In [2]:
import sys
sys.path.append("..")
from lunar_spectra_workbench.pipeline import analyze_day

result = analyze_day("../data/raw/20230902")

result.matches.head()
result.calibration


{'a': np.float64(0.24994452181438387),
 'b': np.float64(200.10083849830252),
 'rmse_nm': np.float64(0.4126167460933406),
 'max_error_nm': np.float64(0.7280369640142794),
 'num_points': 34}